## Day 34 Lecture 1 Assignment

In this assignment, we will learn about SVR. We will use the absenteeism at work dataset loaded below and analyze the model generated for this dataset.

In [0]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
absent = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Absenteeism_at_work.csv', sep=';')

In [0]:
absent.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2


In [0]:

def get_redundant_pairs(df): 
  '''Get diagonal and lower triangular pairs of correlation matrix''' 
  pairs_to_drop = set() 
  cols = df.columns 
  for i in range(0, df.shape[1]): 
    for j in range(0, i+1): 
      pairs_to_drop.add((cols[i], cols[j])) 
  return pairs_to_drop

def get_top_abs_correlations(df, n=5): 
  au_corr = df.corr().abs().unstack() 
  labels_to_drop = get_redundant_pairs(df) 
  au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False) 
  return au_corr[0:n]

get_redundant_pairs(absent)


print("Top Absolute Correlations") 
print(get_top_abs_correlations(absent, n=10))

Top Absolute Correlations
Weight                           Body mass index                    0.904117
Service time                     Age                                0.670979
Reason for absence               Disciplinary failure               0.545054
Service time                     Body mass index                    0.499718
ID                               Distance from Residence to Work    0.486160
Age                              Body mass index                    0.470688
Month of absence                 Hit target                         0.460453
Service time                     Weight                             0.455975
Distance from Residence to Work  Social drinker                     0.452196
ID                               Social drinker                     0.451338
dtype: float64


Recall that we removed all correlated columns as well as columns that are a function of two or more other columns (like BMI and weight and height) as well as columns that are not usefu to us (like ID).
We also created dummy variables for the categorical columns in our dataset.
Finally, we split the data into test and train (with test being 20% of the data).

Perform these actions below.

In [0]:
#answer below:
absent.info()
absent.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               740 non-null    int64  
 1   Reason for absence               740 non-null    int64  
 2   Month of absence                 740 non-null    int64  
 3   Day of the week                  740 non-null    int64  
 4   Seasons                          740 non-null    int64  
 5   Transportation expense           740 non-null    int64  
 6   Distance from Residence to Work  740 non-null    int64  
 7   Service time                     740 non-null    int64  
 8   Age                              740 non-null    int64  
 9   Work load Average/day            740 non-null    float64
 10  Hit target                       740 non-null    int64  
 11  Disciplinary failure             740 non-null    int64  
 12  Education             

Index(['ID', 'Reason for absence', 'Month of absence', 'Day of the week',
       'Seasons', 'Transportation expense', 'Distance from Residence to Work',
       'Service time', 'Age', 'Work load Average/day ', 'Hit target',
       'Disciplinary failure', 'Education', 'Son', 'Social drinker',
       'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index',
       'Absenteeism time in hours'],
      dtype='object')

In [0]:
absdrop = absent.drop(['ID','Height','Hit target'], axis =1)
absdrop
dummies = ['Month of absence', 'Day of the week',
       'Seasons','Education', 'Son']
absdum = pd.DataFrame()
for x in dummies:
  absdum = pd.concat([absdum, pd.get_dummies(absdrop[x], prefix = x, drop_first = True)], axis = 1)

df = pd.concat([absdum, absdrop], axis = 1).drop(dummies, axis=1)
df

,Month of absence_1,Month of absence_2,Month of absence_3,Month of absence_4,Month of absence_5,Month of absence_6,Month of absence_7,Month of absence_8,Month of absence_9,Month of absence_10,Month of absence_11,Month of absence_12,Day of the week_3,Day of the week_4,Day of the week_5,Day of the week_6,Seasons_2,Seasons_3,Seasons_4,Education_2,Education_3,Education_4,Son_1,Son_2,Son_3,Son_4,Reason for absence,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Disciplinary failure,Social drinker,Social smoker,Pet,Weight,Body mass index,Absenteeism time in hours
0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,26,289,36,13,33,239.554,0,1,0,1,90,30,4
1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,118,13,18,50,239.554,1,1,0,0,98,31,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,23,179,51,18,38,239.554,0,1,0,0,89,31,2
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,7,279,5,14,39,239.554,0,1,1,0,68,24,4
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,23,289,36,13,33,239.554,0,1,0,1,90,30,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,14,289,36,13,33,264.604,0,1,0,1,90,30,8
736,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,11,235,11,14,37,264.604,0,0,0,1,88,29,4
737,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,118,14,13,40,271.219,0,1,0,8,98,34,0
738,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,231,35,14,39,271.219,0,1,0,2,100,35,0


Split the data into train and test (test proportion=0.2).

In [0]:
# answer below:
from sklearn.model_selection import train_test_split
X = df.drop('Absenteeism time in hours', axis =1)
y = df['Absenteeism time in hours']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)



Scale the indepdendent variables using the standard scaler

In [0]:
# answer below:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(X_train)
X_train_scale = scaler.transform(X_train)
print(len(X_train))
print(len(X_train_scale))
print(len(y_train))
y_train.isnull().sum()
type(X_train_scale)

592
592
592


numpy.ndarray

In [0]:
C= [10**x for x in range(-5,5)]
C

[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

Generate an SVR model to predict the number of hours of absenteeism. Compare the train and test scores.

In [0]:
# answer below:
from sklearn.svm import SVR
#c = [10**x for x in range(-5,5)]
svrlin = SVR(kernel = 'linear', C= 1000)
svrlin.fit(X_train_scale, y_train)

y_pred_trainlin = svrpoly.predict(X_train)
y_pred_testlin = svrpoly.predict(X_test)


X_test_scale = scaler.transform(X_test)
print(svrlin.score(X_train_scale, y_train))
print(svrlin.score(X_test_scale, y_test))
print('*****************')
print(mean_squared_error(y_pred_trainlin, y_train))
print(mean_squared_error(y_pred_testlin, y_test))

0.05482504971242319
0.07427470983427675
*****************
14745631002759.215
14178790284616.875


In [0]:
# answer below:
from sklearn.svm import SVR
#c = [10**x for x in range(-5,5)]
svrpoly = SVR(kernel = 'rbf', C= .1)
svrpoly.fit(X_train_scale, y_train)

y_pred_trainpoly = svrpoly.predict(X_train)
y_pred_testpoly = svrpoly.predict(X_test)

print(svrpoly.score(X_train_scale, y_train))
print(svrpoly.score(X_test_scale, y_test))
print('*****************')
print(np.sqrt(mean_squared_error(y_pred_trainpoly, y_train)))
print(np.sqrt(mean_squared_error(y_pred_testpoly, y_test)))



-0.05822151917441465
-0.06887751979139645
*****************
14.280815714200727
11.719352990294247


In [0]:
y_train_pred = svrlin.predict(X_train_scale)
import seaborn as sns
sns.scatterplot(y_train, y_train_predlin)


NameError: ignored

In [0]:
sns.scatterplot()

If you are unhappy with your model's performance, perform a grid search to optimize C. You may also want to compare something other than R squared, like MSE, and could try some other kernels than just linear.

In [0]:
# answer below:



In [0]:
from sklearn.datasets import fetch_california_housing
X1, y1 = fetch_california_housing(return_X_y = True)
X1

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])